In [ ]:
!pip install "peft==0.2.0"
!pip install "transformers==4.27.1" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
from transformers import AutoModelForSeq2SeqLM
# huggingface hub model id
model_id = "google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_sa

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import math
from collections import Counter
import re

In [ ]:
import pandas as pd

In [ ]:
def get_input_data(data):
  rows_list = []
  for i in range(len(data)):
    data_input = data['input'][i]
    data1 = data['profile'][i]

    category_data = {}
    for item in data1:


        category = item['category']
        doc_id = item['id']
        doc_text = item['text']

        if category not in category_data:
            category_data[category] = {}

        category_data[category][doc_id] = doc_text





    def tokenize(text):
        if text is None:
            return []
        return text.lower().split()

    def calculate_average_length(docs):
        return sum(len(doc) for doc in docs) / len(docs)

    def calculate_idf(docs, term):

        doc_count_with_term = sum(1 for doc in docs if term in doc)
        return math.log((len(docs) - doc_count_with_term + 0.5) / (doc_count_with_term + 0.5) + 1.0)

    def calculate_bm25_score(query, doc, doc_avg_length, k1=1.5, b=0.75):

        score = 0
        for term in query:
            tf = doc.count(term)
            idf = calculate_idf(doc, term)
            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * len(doc) / doc_avg_length)
            score += idf * numerator / denominator
        return score

    def get_most_relevant_text(query, category_dict):

        query_tokens = tokenize(query)


        scores = []

        for doc_id, doc_text in category_dict.items():
            doc_tokens = tokenize(doc_text)
            bm25_score = calculate_bm25_score(query_tokens, doc_tokens, avg_doc_length)
            scores.append((doc_id, bm25_score))


        sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)


        if sorted_scores:
            return category_dict[sorted_scores[0][0]]
        else:
            return None

    all_docs = [text for category_docs in category_data.values() for text in category_docs.values()]
    avg_doc_length = calculate_average_length(all_docs)



    query = ''.join(data_input.split("article:")[1].split("profile")[0])


    str = ''
    for category, category_dict in category_data.items():
        if len(category_dict) > 1:
            result = get_most_relevant_text(query, category_dict)
            str+=(f"the category for the article: {result} is {category} and ")


        elif len(category_dict) == 1:
            for docid,doc_text in category_dict.items():
              str+=(f"the category for the article: {doc_text} is {category} and ")

    str+=data_input
    rows_list.append(str)
  return rows_list




In [ ]:

import pandas as pd
from sklearn.preprocessing import LabelEncoder

def merged(data_outputs,input_rows):

  df_prof = pd.DataFrame(get_input_data(input_rows))



  data_outputs[['id', 'output']] = data_outputs['golds'].apply(pd.Series)

  df = data_outputs.drop('golds', axis=1)
  df_prof['id'] = ['10' + str(i) for i in range(len(df))]


  df_prof['id'] = df_prof['id'].astype('int64')
  df['id'] = df['id'].astype('int64')
  merged_df = pd.merge(df, df_prof, on='id', how='inner')

  merged_df.rename( columns={0 :'prompt'}, inplace=True )
  merged_df.info(verbose=True)
  return merged_df




In [ ]:
data_train = pd.read_json('https://ciir.cs.umass.edu/downloads/LaMP/LaMP_2/train/train_questions.json')
data_output_train = pd.read_json('https://ciir.cs.umass.edu/downloads/LaMP/LaMP_2/train/train_outputs.json')
# data_test = pd.read_json('https://ciir.cs.umass.edu/downloads/LaMP/LaMP_2/test/test_questions.json')
final_df_train = merged(data_output_train,data_train)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 5914 entries, 0 to 5913
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   task    5914 non-null   object
 1   id      5914 non-null   int64 
 2   output  5914 non-null   object
 3   prompt  5914 non-null   object
dtypes: int64(1), object(3)
memory usage: 231.0+ KB


In [ ]:
def merged_eval(data_outputs,input_rows):

  df_prof = pd.DataFrame(get_input_data(input_rows))



  data_outputs[['id', 'output']] = data_outputs['golds'].apply(pd.Series)

  df = data_outputs.drop('golds', axis=1)
  df_prof['id'] = ['11' + str(i) for i in range(len(df))]

  df_prof['id'] = df_prof['id'].astype('int64')
  df['id'] = df['id'].astype('int64')
  merged_df = pd.merge(df, df_prof, on='id', how='inner')

  merged_df.rename( columns={0 :'prompt'}, inplace=True )
  print (merged_df)
  merged_df.info(verbose=True)
  return merged_df


In [ ]:
def preprocess(input_df):

  model_inputs = tokenizer(input_df['prompt'], max_length=512, truncation=True)
  labels = tokenizer(text_target=input_df['output'], max_length=512, truncation=True)
  model_inputs['labels'] = labels['input_ids']

  return model_inputs

In [ ]:
final_df_train = merged(data_output_train,data_train)
final_df_train.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5914 entries, 0 to 5913
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   task    5914 non-null   object
 1   id      5914 non-null   int64 
 2   output  5914 non-null   object
 3   prompt  5914 non-null   object
dtypes: int64(1), object(3)
memory usage: 231.0+ KB


,task,id,output,prompt
0,LaMP_2,100,business,the category for the article: The three make a...
1,LaMP_2,101,style & beauty,the category for the article: By Paul Chi Netf...
2,LaMP_2,102,style & beauty,the category for the article: By Paul Chi Netf...
3,LaMP_2,103,entertainment,the category for the article: By Paul Chi Netf...
4,LaMP_2,104,style & beauty,the category for the article: By Paul Chi Netf...
5,LaMP_2,105,style & beauty,the category for the article: By Paul Chi Netf...
6,LaMP_2,106,politics,the category for the article: Cybercrime is ev...
7,LaMP_2,107,politics,the category for the article: What do you teac...
8,LaMP_2,108,sports,the category for the article: What do you teac...
9,LaMP_2,109,sports,the category for the article: What do you teac...


In [ ]:
from datasets import Dataset
final_df_train = Dataset.from_pandas(final_df_train)
# tokenzied_data = final_df_train.map(preprocess)


In [ ]:
tokenzied_data = final_df_train.map(preprocess)

  0%|          | 0/5914 [00:00<?, ?ex/s]

In [ ]:
data_validate = pd.read_json('https://ciir.cs.umass.edu/downloads/LaMP/LaMP_2/dev/dev_questions.json')
data_output_validate = pd.read_json('https://ciir.cs.umass.edu/downloads/LaMP/LaMP_2/dev/dev_outputs.json')


final_df_evaluate = merged_eval(data_output_validate,data_validate)

        task      id                output  \
0     LaMP_2     110              politics   
1     LaMP_2     111              politics   
2     LaMP_2     112              politics   
3     LaMP_2     113                sports   
4     LaMP_2     114                 crime   
...      ...     ...                   ...   
1047  LaMP_2  111047         entertainment   
1048  LaMP_2  111048              religion   
1049  LaMP_2  111049                travel   
1050  LaMP_2  111050  science & technology   
1051  LaMP_2  111051              politics   

                                                 prompt  
0     the category for the article: The U.S. has the...  
1     the category for the article: Her unconvention...  
2     the category for the article: Her unconvention...  
3     the category for the article: Pres. Donald Tru...  
4     the category for the article: The dismissed su...  
...                                                 ...  
1047  the category for the article: We ca

In [ ]:
final_df_evaluate = Dataset.from_pandas(final_df_evaluate)
tokenzied_data_eval = final_df_evaluate.map(preprocess)

  0%|          | 0/1052 [00:00<?, ?ex/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="./results"
L_RATE = 5 * 10e-5
BATCH_SIZE = 16
PER_DEVICE_EVAL_BATCH = 8
WEIGHT_DECAY = 0.001
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 6

total_train_steps = len(final_df_train)// BATCH_SIZE * NUM_EPOCHS
warmup_percentage = 0.05

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    push_to_hub=False,
    lr_scheduler_type='linear',
    warmup_steps=int(total_train_steps * warmup_percentage)
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenzied_data,
    eval_dataset=tokenzied_data_eval,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/l

Epoch,Training Loss,Validation Loss
1,No log,0.344163
2,0.735900,0.360146
3,0.488800,0.346965
4,0.488800,0.345259
5,0.448600,0.345801
6,0.416000,0.340599


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain 

TrainOutput(global_step=2220, training_loss=0.5105427888062624, metrics={'train_runtime': 5515.646, 'train_samples_per_second': 6.433, 'train_steps_per_second': 0.402, 'total_flos': 2.449046936302387e+16, 'train_loss': 0.5105427888062624, 'epoch': 6.0})

In [ ]:
outputs = trainer.predict(tokenzied_data_eval)

In [ ]:
output_decoded = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)
output_decoded

['politics',
 'politics',
 'politics',
 'politics',
 'crime',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'sports',
 'entertainment',
 'entertainment',
 'style & beauty',
 'sports',
 'style & beauty',
 'business',
 'style & beauty',
 'travel',
 'women',
 'healthy living',
 'style & beauty',
 'healthy living',
 'healthy living',
 'healthy living',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'sports',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'business',
 'politics',
 'politics',
 'business',
 'politics',
 'politics',
 'entertainment',
 'entertainment',
 'entertainment',
 'politics',
 'entertainment',
 'entertainment',
 'entertainment',
 'e

In [ ]:
labels = list(final_df_evaluate['output'])


In [ ]:
correct = 0
for i in range(len(output_decoded)):
  if output_decoded[i] == labels[i]:
    correct += 1

print("Accuracy of Val Data:", (correct / len(labels)) * 100)

Accuracy of Val Data: 80.03802281368822


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(labels, output_decoded)
print("Accuracy of Val Data:", accuracy * 100)
f1 = f1_score(labels, output_decoded, average='weighted')
print("F1 Score of Val Data:", f1)

Accuracy of Val Data: 80.03802281368822
F1 Score of Val Data: 0.7903828286664735
